In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import numpy as np
from Models import MoELSTM
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

from typing import List, Tuple, Optional, Dict
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
import random
from Models import MoELSTM, LSTMModel, train_model
from Preprocess import (
    compute_metrics,
    convert_timeseries_to_numpy,
    create_dataloader,
    load_building_series,
    split_series_list,
)
import pandas as pd
from collections import defaultdict
import os
import torch
import torch.optim as optim
from tqdm import tqdm


from Models import model_fn
from tqdm import tqdm
from my_utils import train_model, load_energy_data_feather, get_weights, set_weights


In [22]:
from AggregationStrategy import sync_aggregate,average_weights,sync_aggregate_norm,sync_aggregate_softmax, fedavgm_update

In [23]:
# def average_weights(weights_list):
#     """Averages model weights provided as a list of get_weights outputs."""
#     avg_weights = []
#     num_models = len(weights_list)
#     for layer_weights in zip(*weights_list):
#         avg_layer = np.mean(np.array(layer_weights), axis=0)
#         avg_weights.append(avg_layer)
#     return avg_weights

In [24]:
df = pd.read_feather("train_final.feather")

In [25]:
df.head()

,building_id,meter,timestamp,meter_reading,primary_use,air_temperature
7593144,0,0,2016-05-21 01:00:00,72.221012,Education,25.6
7593145,1,0,2016-05-21 01:00:00,39.611586,Education,25.6
7593146,2,0,2016-05-21 01:00:00,1.920567,Education,25.6
7593147,3,0,2016-05-21 01:00:00,111.532464,Education,25.6
7593148,4,0,2016-05-21 01:00:00,456.734799,Education,25.6


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11712248 entries, 7593144 to 20216099
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   building_id      int64         
 1   meter            int64         
 2   timestamp        datetime64[ns]
 3   meter_reading    float64       
 4   primary_use      object        
 5   air_temperature  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 625.5+ MB


In [27]:

def cluster_buildings_top3_primary_use(df: pd.DataFrame) -> dict:
    """
    Clusters building_ids by top 3 primary_use categories.
    Remaining categories are grouped into 'Other'.

    Args:
        df (pd.DataFrame): Input DataFrame with 'building_id' and 'primary_use'.

    Returns:
        dict: {'cluster_0': [...], 'cluster_1': [...], 'cluster_2': [...], 'other': [...]}
    """
    # Step 1: Get top 3 most common primary_use values
    top3_uses = df['primary_use'].value_counts().nlargest(3).index.tolist()

    # Step 2: Create a mapping of cluster name -> list of building_ids
    clusters = defaultdict(list)

    # Drop duplicate building_id-primary_use pairs to avoid counting duplicates
    unique_buildings = df[['building_id', 'primary_use']].drop_duplicates()

    for _, row in unique_buildings.iterrows():
        bldg_id = row['building_id']
        use = row['primary_use']
        if use == top3_uses[0]:
            clusters['cluster_0'].append(bldg_id)
        elif use == top3_uses[1]:
            clusters['cluster_1'].append(bldg_id)
        elif use == top3_uses[2]:
            clusters['cluster_2'].append(bldg_id)
        else:
            clusters['other'].append(bldg_id)

    return dict(clusters)


In [28]:
top3_uses = df['primary_use'].value_counts().nlargest(3).index.tolist()

In [29]:
top3_uses

['Education', 'Office', 'Entertainment/public assembly']

In [30]:
clusters = cluster_buildings_top3_primary_use(df)

for name, ids in clusters.items():
    print(f"{name}: {len(ids)} buildings {ids[:5]}...")


cluster_0: 537 buildings [0, 1, 2, 3, 4]...
other: 428 buildings [6, 12, 27, 33, 34]...
cluster_1: 269 buildings [9, 15, 17, 19, 21]...
cluster_2: 179 buildings [10, 59, 87, 88, 40]...


In [31]:


# Config
# List of models to experiment with
MODEL_NAMES = ["lstm", "gru", "moe_lstm", "moe_gru"]

# Config
NUM_CLIENTS = 1410
CLIENT_FRAC = 0.15
NUM_ROUNDS = 50
LOCAL_EPOCHS = 5
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_FILE ="train_final.feather" # "meter_0_data_cleaned.feather"


### Naive Model :
Returns values by lag-24 hr (Acts as Lower Bound)

### Global Models
All data, no Fedeerated Aggregation

In [15]:
for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)
    global_model = model_fn(model_name).to(DEVICE)

    sampled_clients = list(range(NUM_CLIENTS))
    

    for cid in tqdm(sampled_clients, desc="Training clients"):
        
        train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

        updated_weights, fin_loss = train_model(
            global_model, train_loader,
            device=DEVICE,
            learning_rate=LR,
            loss_fn=None,
            optimizer_class=optim.Adam,
            epochs=LOCAL_EPOCHS
        )
    checkpoint_path = os.path.join(model_dir, f"{model_name}_global_model.pt")
    torch.save(global_model.state_dict(), checkpoint_path)
    print(f"Saved global model to {checkpoint_path}")

Starting experiment with model: lstm


Training clients: 100%|██████████| 1410/1410 [10:34<00:00,  2.22it/s]


Saved global model to results/lstm/lstm_global_model.pt
Starting experiment with model: gru


Training clients: 100%|██████████| 1410/1410 [10:28<00:00,  2.24it/s]


Saved global model to results/gru/gru_global_model.pt
Starting experiment with model: moe_lstm


Training clients: 100%|██████████| 1410/1410 [12:42<00:00,  1.85it/s]


Saved global model to results/moe_lstm/moe_lstm_global_model.pt
Starting experiment with model: moe_gru


Training clients: 100%|██████████| 1410/1410 [12:38<00:00,  1.86it/s]

Saved global model to results/moe_gru/moe_gru_global_model.pt


## Federated Learning Without Clustering

### FedAvg

In [ ]:

for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, fin_loss = train_model(
                local_model, train_loader,
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

        # Federated averaging
        global_weights = average_weights(local_weights)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_fedAvg_lr.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: lstm
Round 1/50


Training clients:  24%|██▎       | 50/211 [00:22<01:12,  2.22it/s]

### FedAvg-diff

In [32]:
import numpy as np

# Initialize difficulty scores and sampling probabilities
difficulty_scores = np.ones(NUM_CLIENTS)
sampling_probs = np.ones(NUM_CLIENTS) / NUM_CLIENTS  # Start uniform

for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        
        # Sample clients using difficulty-aware probability
        sampled_clients = np.random.choice(
            NUM_CLIENTS,
            size=int(CLIENT_FRAC * NUM_CLIENTS),
            replace=False,
            p=sampling_probs
        ).tolist()

        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, loss_history = train_model(
                local_model, train_loader,
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

            # === Update difficulty score ===
            final_loss = loss_history[-1]  # Or use a more complex difficulty tracker
            # Optionally apply EMA smoothing
            alpha = 0.9
            difficulty_scores[cid] = alpha * difficulty_scores[cid] + (1 - alpha) * final_loss

        # Update sampling probabilities: lower prob for higher difficulty
        inv_difficulty = 1.0 / (np.array(difficulty_scores) + 1e-8)
        sampling_probs = inv_difficulty / inv_difficulty.sum()

        # Federated averaging
        global_weights = average_weights(local_weights)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_fedAvg_diffsample.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: lstm
Round 1/50


Training clients: 100%|██████████| 211/211 [01:35<00:00,  2.22it/s]


Saved global model to results/lstm/lstm_round_1_fedAvg_diffsample.pt
Round 2/50


Training clients: 100%|██████████| 211/211 [01:36<00:00,  2.20it/s]


Saved global model to results/lstm/lstm_round_2_fedAvg_diffsample.pt
Round 3/50


...

Saved global model to results/moe_gru/moe_gru_round_48_fedAvg_diffsample.pt
Round 49/50


Training clients: 100%|██████████| 211/211 [02:01<00:00,  1.73it/s]


Saved global model to results/moe_gru/moe_gru_round_49_fedAvg_diffsample.pt
Round 50/50


Training clients: 100%|██████████| 211/211 [02:03<00:00,  1.70it/s]

Saved global model to results/moe_gru/moe_gru_round_50_fedAvg_diffsample.pt


### FedAvgM 

In [ ]:

for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    velocity = [np.zeros_like(w) for w in global_weights]

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, fin_loss = train_model(
                local_model, train_loader,
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

        # Federated averaging
        global_weights, velocity = fedavgm_update(global_weights,local_weights,velocity)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_fedAvgM.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: lstm
Round 1/10


Training clients: 100%|██████████| 211/211 [01:35<00:00,  2.21it/s]


Saved global model to results/lstm/lstm_round_1_fedAvgM.pt
Round 2/10


Training clients: 100%|██████████| 211/211 [01:36<00:00,  2.19it/s]


Saved global model to results/lstm/lstm_round_2_fedAvgM.pt
Round 3/10


...

Training clients: 100%|██████████| 211/211 [01:36<00:00,  2.19it/s]


Saved global model to results/lstm/lstm_round_10_fedAvgM.pt
Starting experiment with model: gru
Round 1/10


Training clients: 100%|██████████| 211/211 [01:34<00:00,  2.22it/s]


Saved global model to results/gru/gru_round_1_fedAvgM.pt
Round 2/10


Training clients:  79%|███████▊  | 166/211 [01:14<00:21,  2.09it/s]

### FedProx

In [13]:
def train_model_fedprox(
    model,
    train_loader,
    global_weights,
    device,
    learning_rate,
    loss_fn,
    optimizer_class,
    epochs,
    mu=0.01
):
    model.train()
    model.to(device)
    optimizer = optimizer_class(model.parameters(), lr=learning_rate)
    loss_fn = loss_fn or torch.nn.MSELoss()

    original_weights = [w.clone().detach().to(device) for w in model.parameters()]
    loss_history = []

    for _ in range(epochs):
        for batch in train_loader:
            optimizer.zero_grad()
            x, y = batch[0].to(device), batch[1].to(device)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)

            # FedProx proximal term
            prox_term = 0.0
            for param, orig in zip(model.parameters(), global_weights):
                prox_term += ((param - orig.to(device)) ** 2).sum()
            prox_term *= (mu / 2)

            total_loss = loss + prox_term
            total_loss.backward()
            optimizer.step()

            loss_history.append(total_loss.item())

    return get_weights(model), loss_history


In [16]:
def train_model_fedprox(
    model,
    train_loader,
    global_weights,
    device=None,
    learning_rate=0.001,
    loss_fn=None,
    optimizer_class=optim.Adam,
    epochs=50,
    mu=0.01
):
    """Train model using FedProx loss (with proximal term)."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    loss_fn = loss_fn or nn.MSELoss()
    optimizer = optimizer_class(model.parameters(), lr=learning_rate)
    loss_history = []

    # Clone global weights as tensors on the correct device
    # prox_ref = [torch.tensor(w, dtype=torch.float32, device=device) for w in global_weights]
    prox_ref = [w.detach().clone().to(device) for w in global_weights]

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            if y_batch.dim() == 3 and y_batch.shape[-1] == 1:
                y_batch = y_batch.squeeze(-1)

            optimizer.zero_grad()
            output = model(X_batch)
            task_loss = loss_fn(output, y_batch)

            # Compute proximal term
            prox_term = 0.0
            for p, w_ref in zip(model.parameters(), prox_ref):
                prox_term += torch.norm(p - w_ref) ** 2
            prox_term = (mu / 2) * prox_term

            total_loss = task_loss + prox_term
            total_loss.backward()
            optimizer.step()

            epoch_loss += total_loss.item()

        loss_history.append(epoch_loss / len(train_loader))

    return get_weights(model), loss_history

In [17]:
for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, loss_history = train_model_fedprox(
                model=local_model,
                train_loader=train_loader,
                global_weights=[torch.tensor(w).to(DEVICE) for w in global_weights],
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS,
                mu=0.01  # optional, default used if not passed
            )
            local_weights.append(updated_weights)

        # FedProx uses FedAvg-style aggregation
        global_weights = average_weights(local_weights)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_fedProx.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: lstm
Round 1/10


Training clients: 100%|██████████| 211/211 [02:46<00:00,  1.27it/s]


Saved global model to results/lstm/lstm_round_1_fedProx.pt
Round 2/10


Training clients: 100%|██████████| 211/211 [02:51<00:00,  1.23it/s]


Saved global model to results/lstm/lstm_round_2_fedProx.pt
Round 3/10


...

Saved global model to results/moe_gru/moe_gru_round_8_fedProx.pt
Round 9/10


Training clients: 100%|██████████| 211/211 [03:30<00:00,  1.00it/s]


Saved global model to results/moe_gru/moe_gru_round_9_fedProx.pt
Round 10/10


Training clients: 100%|██████████| 211/211 [03:32<00:00,  1.01s/it]

Saved global model to results/moe_gru/moe_gru_round_10_fedProx.pt


### Kuramoto FedAvg

In [ ]:

for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, fin_loss = train_model(
                local_model, train_loader,
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

        # Federated averaging
        global_weights = sync_aggregate(global_weights,local_weights)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_kr.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


### Kuramoto-Norm FedAvg

In [ ]:

for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, fin_loss = train_model(
                local_model, train_loader,
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

        # Federated averaging
        global_weights = sync_aggregate_norm(global_weights,local_weights)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_kr_norm.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


### Kuramoto-Softmax

In [ ]:

for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, fin_loss = train_model(
                local_model, train_loader,
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

        # Federated averaging
        global_weights = sync_aggregate_softmax(global_weights,local_weights)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_kr_sft.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


### DiffAware FedAvg

In [12]:
class TimeSeriesDifficultyWeight:
    def __init__(self, num_clients, accumulate_iters=20):
        self.num_clients = num_clients
        self.last_loss = torch.ones(num_clients).float().to(DEVICE)
        self.learn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.unlearn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.ema_difficulty = torch.ones(num_clients).float().to(DEVICE)
        self.accumulate_iters = accumulate_iters

    def update(self, cid: int, loss_history: List[float]) -> float:
        """
        Update difficulty based on loss trend for a client.
        Expects a list of per-epoch losses.
        """
        current_loss = torch.tensor(loss_history[-1], dtype=torch.float32).to(DEVICE)
        previous_loss = self.last_loss[cid]
        delta = current_loss - previous_loss
        ratio = torch.log((current_loss + 1e-8) / (previous_loss + 1e-8))

        learn = torch.where(delta < 0, -delta * ratio, torch.tensor(0.0, device=current_loss.device))
        unlearn = torch.where(delta >= 0, delta * ratio, torch.tensor(0.0, device=current_loss.device))

        # EMA update
        momentum = (self.accumulate_iters - 1) / self.accumulate_iters
        self.learn_score[cid] = momentum * self.learn_score[cid] + (1 - momentum) * learn
        self.unlearn_score[cid] = momentum * self.unlearn_score[cid] + (1 - momentum) * unlearn

        # Difficulty score
        diff_ratio = (self.unlearn_score[cid] + 1e-8) / (self.learn_score[cid] + 1e-8)
        difficulty = diff_ratio #torch.pow(diff_ratio, 1 / 5)

        # Smooth difficulty over rounds
        self.ema_difficulty[cid] = momentum * self.ema_difficulty[cid] + (1 - momentum) * difficulty

        self.last_loss[cid] = current_loss
        return self.ema_difficulty[cid].item()

    def get_normalized_weights(self, client_ids: List[int]) -> List[float]:
        weights = [self.ema_difficulty[cid].item() for cid in client_ids]
        total = sum(weights)
        if total == 0:
            return [1.0 / len(client_ids)] * len(client_ids)
        return [w / total for w in weights]


In [14]:
difficulty_tracker = TimeSeriesDifficultyWeight(num_clients=NUM_CLIENTS)

for rnd in range(NUM_ROUNDS):
    sampled_clients = random.sample(range(NUM_CLIENTS), int(CLIENT_FRAC * NUM_CLIENTS))
    local_weights = []
    difficulty_scores = []

    for cid in tqdm(sampled_clients):
        model = model_fn(model_name).to(DEVICE)
        set_weights(model, global_weights)
        train_loader, _ = load_energy_data_feather(cid, filepath=DATA_FILE)
        updated_weights, loss_history = train_model(
            model, train_loader,
            device=DEVICE, learning_rate=LR,
            loss_fn=None, optimizer_class=optim.Adam,
            epochs=LOCAL_EPOCHS
        )
        local_weights.append(updated_weights)

        # Update difficulty
        difficulty = difficulty_tracker.update(cid, loss_history)
        difficulty_scores.append(difficulty)

    # Normalize difficulty scores
    normalized_weights = difficulty_tracker.get_normalized_weights(sampled_clients)

    # Difficulty-aware weighted aggregation
    global_weights = average_weights(local_weights, client_weights=normalized_weights)
    set_weights(global_model, global_weights)


  0%|          | 0/211 [00:00<?, ?it/s]


NameError: name 'model_name' is not defined

## Federated Learning With Clustering

In [14]:
import os
import torch
import torch.optim as optim
from tqdm import tqdm
import random
import numpy as np

CLUSTERS = cluster_buildings_top3_primary_use(df)

for model_name in MODEL_NAMES:
    for cluster_name in CLUSTERS:
        os.makedirs(os.path.join("results", model_name, cluster_name), exist_ok=True)


### FedAvg

In [ ]:

# Main experiment loop
for model_name in MODEL_NAMES:
    print(f"Starting experiments for model: {model_name}")

    # Initialize per-cluster model weights
    cluster_models = {}
    cluster_weights = {}

    for cluster_name in CLUSTERS:
        model = model_fn(model_name).to(DEVICE)
        cluster_models[cluster_name] = model
        cluster_weights[cluster_name] = get_weights(model)

    for rnd in range(NUM_ROUNDS):
        print(f"\Round {rnd+1}/{NUM_ROUNDS}")

        for cluster_name, client_ids in CLUSTERS.items():
            print(f" Processing {cluster_name} with {len(client_ids)} clients")

            # Sample a fraction of clients from the cluster
            sampled_clients = random.sample(client_ids, k=int(CLIENT_FRAC * len(client_ids)))
            print(f"Sampling {len(sampled_clients)} Clients")
            local_weights = []

            for cid in tqdm(sampled_clients, desc=f"Training {cluster_name}"):
                local_model = model_fn(model_name).to(DEVICE)
                set_weights(local_model, cluster_weights[cluster_name])

                train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)
                updated_weights, fin_loss = train_model(
                    local_model, train_loader,
                    device=DEVICE,
                    learning_rate=LR,
                    loss_fn=None,
                    optimizer_class=optim.Adam,
                    epochs=LOCAL_EPOCHS
                )
                local_weights.append(updated_weights)

            # Aggregate and update cluster model
            updated_cluster_weights = average_weights(local_weights)
            set_weights(cluster_models[cluster_name], updated_cluster_weights)
            cluster_weights[cluster_name] = updated_cluster_weights

            # Save checkpoint
            ckpt_path = os.path.join("results", model_name, cluster_name, f"{model_name}_{cluster_name}_round_{rnd+1}.pt")
            torch.save(cluster_models[cluster_name].state_dict(), ckpt_path)
            print(f"Saved model: {ckpt_path}")


### FedAvgM

In [15]:
import os
import torch
import torch.optim as optim
from tqdm import tqdm

# Main experiment loop
for model_name in MODEL_NAMES:
    print(f"Starting experiments for model: {model_name}")

    # Initialize per-cluster model weights and velocity
    cluster_models = {}
    cluster_weights = {}
    cluster_velocities = {}

    for cluster_name in CLUSTERS:
        model = model_fn(model_name).to(DEVICE)
        weights = get_weights(model)
        velocity = [np.zeros_like(w) for w in weights]

        cluster_models[cluster_name] = model
        cluster_weights[cluster_name] = weights
        cluster_velocities[cluster_name] = velocity

    for rnd in range(NUM_ROUNDS):
        print(f"\nRound {rnd+1}/{NUM_ROUNDS}")

        for cluster_name, client_ids in CLUSTERS.items():
            print(f" Processing {cluster_name} with {len(client_ids)} clients")

            # Sample a fraction of clients from the cluster
            sampled_clients = random.sample(client_ids, k=int(CLIENT_FRAC * len(client_ids)))
            print(f"Sampling {len(sampled_clients)} Clients")
            local_weights = []

            for cid in tqdm(sampled_clients, desc=f"Training {cluster_name}"):
                local_model = model_fn(model_name).to(DEVICE)
                set_weights(local_model, cluster_weights[cluster_name])

                train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)
                updated_weights, fin_loss = train_model(
                    local_model, train_loader,
                    device=DEVICE,
                    learning_rate=LR,
                    loss_fn=None,
                    optimizer_class=optim.Adam,
                    epochs=LOCAL_EPOCHS
                )
                local_weights.append(updated_weights)

            # ---- FedAvgM Aggregation ----
            new_weights, new_velocity = fedavgm_update(
                cluster_weights[cluster_name],
                local_weights,
                cluster_velocities[cluster_name]
            )

            # Update model, weights, and velocity
            set_weights(cluster_models[cluster_name], new_weights)
            cluster_weights[cluster_name] = new_weights
            cluster_velocities[cluster_name] = new_velocity

            # Save checkpoint
            ckpt_dir = os.path.join("results", model_name, cluster_name)
            os.makedirs(ckpt_dir, exist_ok=True)
            ckpt_path = os.path.join(ckpt_dir, f"{model_name}_{cluster_name}_round_{rnd+1}_fedAvgM.pt")
            torch.save(cluster_models[cluster_name].state_dict(), ckpt_path)
            print(f"Saved model: {ckpt_path}")


Starting experiments for model: lstm

Round 1/10
 Processing cluster_0 with 537 clients
Sampling 80 Clients


Training cluster_0: 100%|██████████| 80/80 [00:35<00:00,  2.23it/s]


Saved model: results/lstm/cluster_0/lstm_cluster_0_round_1_fedAvgM.pt
 Processing other with 428 clients
Sampling 64 Clients


Training other: 100%|██████████| 64/64 [00:28<00:00,  2.22it/s]


Saved model: results/lstm/other/lstm_other_round_1_fedAvgM.pt
 Processing cluster_1 with 269 clients
Sampling 40 Clients


...

Saved model: results/moe_gru/other/moe_gru_other_round_10_fedAvgM.pt
 Processing cluster_1 with 269 clients
Sampling 40 Clients


Training cluster_1: 100%|██████████| 40/40 [00:20<00:00,  1.95it/s]


Saved model: results/moe_gru/cluster_1/moe_gru_cluster_1_round_10_fedAvgM.pt
 Processing cluster_2 with 179 clients
Sampling 26 Clients


Training cluster_2: 100%|██████████| 26/26 [00:13<00:00,  1.92it/s]

Saved model: results/moe_gru/cluster_2/moe_gru_cluster_2_round_10_fedAvgM.pt


### FedAdam

In [17]:
from AggregationStrategy import fed_adam_update

In [18]:
from tqdm import tqdm
import os
import random
import torch

# Main experiment loop using FedAdam
for model_name in MODEL_NAMES:
    print(f"Starting experiments for model: {model_name}")

    cluster_models = {}
    cluster_weights = {}
    cluster_m_t = {}
    cluster_v_t = {}

    # Initialize model, weights, m_t, v_t per cluster
    for cluster_name in CLUSTERS:
        model = model_fn(model_name).to(DEVICE)
        weights = get_weights(model)
        cluster_models[cluster_name] = model
        cluster_weights[cluster_name] = weights

        # Initialize m_t and v_t as zero arrays matching model shape
        cluster_m_t[cluster_name] = [np.zeros_like(w) for w in weights]
        cluster_v_t[cluster_name] = [np.zeros_like(w) for w in weights]

    # Federated training rounds
    for rnd in range(NUM_ROUNDS):
        print(f"\nRound {rnd + 1}/{NUM_ROUNDS}")

        for cluster_name, client_ids in CLUSTERS.items():
            print(f" Processing {cluster_name} with {len(client_ids)} clients")

            sampled_clients = random.sample(client_ids, k=int(CLIENT_FRAC * len(client_ids)))
            print(f" Sampling {len(sampled_clients)} Clients")
            local_weights = []

            for cid in tqdm(sampled_clients, desc=f"Training {cluster_name}"):
                local_model = model_fn(model_name).to(DEVICE)
                set_weights(local_model, cluster_weights[cluster_name])

                train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)
                updated_weights, _ = train_model(
                    local_model, train_loader,
                    device=DEVICE,
                    learning_rate=LR,
                    loss_fn=None,
                    optimizer_class=optim.Adam,
                    epochs=LOCAL_EPOCHS
                )
                local_weights.append(updated_weights)

            # Perform FedAdam update
            updated_weights, new_m_t, new_v_t = fed_adam_update(
                global_weights=cluster_weights[cluster_name],
                local_weights_list=local_weights,
                m_t=cluster_m_t[cluster_name],
                v_t=cluster_v_t[cluster_name],
                eta=0.001,       # or your server LR
                beta1=0.9,
                beta2=0.999,
                eps=1e-8
            )

            # Update weights and moments
            set_weights(cluster_models[cluster_name], updated_weights)
            cluster_weights[cluster_name] = updated_weights
            cluster_m_t[cluster_name] = new_m_t
            cluster_v_t[cluster_name] = new_v_t

            # Save checkpoint
            ckpt_dir = os.path.join("results", model_name, cluster_name)
            os.makedirs(ckpt_dir, exist_ok=True)
            ckpt_path = os.path.join(ckpt_dir, f"{model_name}_{cluster_name}_round_{rnd + 1}_fedAdam.pt")
            torch.save(cluster_models[cluster_name].state_dict(), ckpt_path)
            print(f"Saved model: {ckpt_path}")


Starting experiments for model: lstm

Round 1/10
 Processing cluster_0 with 537 clients
 Sampling 80 Clients


Training cluster_0:   0%|          | 0/80 [00:00<?, ?it/s]

Training cluster_0: 100%|██████████| 80/80 [00:36<00:00,  2.22it/s]


Saved model: results/lstm/cluster_0/lstm_cluster_0_round_1_fedAdam.pt
 Processing other with 428 clients
 Sampling 64 Clients


Training other: 100%|██████████| 64/64 [00:28<00:00,  2.23it/s]


...

Saved model: results/moe_gru/other/moe_gru_other_round_10_fedAdam.pt
 Processing cluster_1 with 269 clients
 Sampling 40 Clients


Training cluster_1: 100%|██████████| 40/40 [00:21<00:00,  1.90it/s]


Saved model: results/moe_gru/cluster_1/moe_gru_cluster_1_round_10_fedAdam.pt
 Processing cluster_2 with 179 clients
 Sampling 26 Clients


Training cluster_2: 100%|██████████| 26/26 [00:13<00:00,  1.88it/s]

Saved model: results/moe_gru/cluster_2/moe_gru_cluster_2_round_10_fedAdam.pt


### Kuramoto FedAvg

In [ ]:

# Main experiment loop
for model_name in MODEL_NAMES:
    print(f"Starting experiments for model: {model_name}")

    # Initialize per-cluster model weights
    cluster_models = {}
    cluster_weights = {}

    for cluster_name in CLUSTERS:
        model = model_fn(model_name).to(DEVICE)
        cluster_models[cluster_name] = model
        cluster_weights[cluster_name] = get_weights(model)

    for rnd in range(NUM_ROUNDS):
        print(f"\Round {rnd+1}/{NUM_ROUNDS}")

        for cluster_name, client_ids in CLUSTERS.items():
            print(f" Processing {cluster_name} with {len(client_ids)} clients")

            # Sample a fraction of clients from the cluster
            sampled_clients = random.sample(client_ids, k=int(CLIENT_FRAC * len(client_ids)))
            print(f"Sampling {len(sampled_clients)} Clients")
            local_weights = []

            for cid in tqdm(sampled_clients, desc=f"Training {cluster_name}"):
                local_model = model_fn(model_name).to(DEVICE)
                set_weights(local_model, cluster_weights[cluster_name])

                train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)
                updated_weights, fin_loss = train_model(
                    local_model, train_loader,
                    device=DEVICE,
                    learning_rate=LR,
                    loss_fn=None,
                    optimizer_class=optim.Adam,
                    epochs=LOCAL_EPOCHS
                )
                local_weights.append(updated_weights)

            # Aggregate and update cluster model
            updated_cluster_weights = sync_aggregate(cluster_weights[cluster_name],local_weights)
            set_weights(cluster_models[cluster_name], updated_cluster_weights)
            cluster_weights[cluster_name] = updated_cluster_weights

            # Save checkpoint
            ckpt_path = os.path.join("results", model_name, cluster_name, f"{model_name}_{cluster_name}_round_{rnd+1}_kr.pt")
            torch.save(cluster_models[cluster_name].state_dict(), ckpt_path)
            print(f"Saved model: {ckpt_path}")


### Kuramoto Softmax FedAvg

In [ ]:

# Main experiment loop
for model_name in MODEL_NAMES:
    print(f"Starting experiments for model: {model_name}")

    # Initialize per-cluster model weights
    cluster_models = {}
    cluster_weights = {}

    for cluster_name in CLUSTERS:
        model = model_fn(model_name).to(DEVICE)
        cluster_models[cluster_name] = model
        cluster_weights[cluster_name] = get_weights(model)

    for rnd in range(NUM_ROUNDS):
        print(f"\Round {rnd+1}/{NUM_ROUNDS}")

        for cluster_name, client_ids in CLUSTERS.items():
            print(f" Processing {cluster_name} with {len(client_ids)} clients")

            # Sample a fraction of clients from the cluster
            sampled_clients = random.sample(client_ids, k=int(CLIENT_FRAC * len(client_ids)))
            print(f"Sampling {len(sampled_clients)} Clients")
            local_weights = []

            for cid in tqdm(sampled_clients, desc=f"Training {cluster_name}"):
                local_model = model_fn(model_name).to(DEVICE)
                set_weights(local_model, cluster_weights[cluster_name])

                train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)
                updated_weights, fin_loss = train_model(
                    local_model, train_loader,
                    device=DEVICE,
                    learning_rate=LR,
                    loss_fn=None,
                    optimizer_class=optim.Adam,
                    epochs=LOCAL_EPOCHS
                )
                local_weights.append(updated_weights)

            # Aggregate and update cluster model
            updated_cluster_weights = sync_aggregate_softmax(cluster_weights[cluster_name],local_weights)
            set_weights(cluster_models[cluster_name], updated_cluster_weights)
            cluster_weights[cluster_name] = updated_cluster_weights

            # Save checkpoint
            ckpt_path = os.path.join("results", model_name, cluster_name, f"{model_name}_{cluster_name}_round_{rnd+1}_kr_softmax.pt")
            torch.save(cluster_models[cluster_name].state_dict(), ckpt_path)
            print(f"Saved model: {ckpt_path}")


Starting experiments for model: lstm
\Round 1/10
 Processing cluster_0 with 537 clients
Sampling 80 Clients


Training cluster_0: 100%|██████████| 80/80 [00:39<00:00,  2.05it/s]


SYNC Weights:[0.01714586280286312, 0.007341246586292982, 0.005376106593757868, 0.009026425890624523, 0.013748370110988617, 0.003524855012074113, 0.015615459531545639, 0.028545405715703964, 0.00037552270805463195, 0.0026444564573466778, 0.015169553458690643, 0.016310110688209534, 0.0002666755754034966, 0.024659018963575363, 0.010934737510979176, 0.005372896324843168, 0.008823621086776257, 0.01801506243646145, 0.03618618845939636, 0.0002804785908665508, 0.007796133868396282, 0.02834409847855568, 0.0006704466068185866, 0.003171334508806467, 0.009118565358221531, 0.013552341610193253, 0.010257434099912643, 0.006262031849473715, 0.025177812203764915, 0.01611536741256714, 0.004576415289193392, 0.0010064546950161457, 0.01566484570503235, 0.006497932132333517, 0.017817247658967972, 0.009647764265537262, 0.018062923103570938, 0.026207640767097473, 0.004003540147095919, 0.020688243210315704, 0.003593906294554472, 0.00176674232352525, 0.0036300714127719402, 0.02171650156378746, 0.0176354553550481

Training other: 100%|██████████| 64/64 [00:32<00:00,  1.99it/s]


SYNC Weights:[3.994156591010501e-25, 1.9981471268692985e-05, 0.000170990388141945, 1.1178276360102046e-10, 2.834427300073966e-13, 0.10563703626394272, 9.04088331976638e-11, 1.2971519254278974e-06, 0.0042074513621628284, 4.567755895745762e-10, 1.4396679860430196e-17, 1.2953025154208153e-12, 2.2268265790353903e-13, 4.590889091105055e-08, 1.1056963111484401e-14, 4.6250628656707704e-05, 4.2638656006562664e-14, 0.02651219815015793, 1.591702675796114e-05, 1.1942501032535802e-07, 8.364056157006416e-06, 2.479996510373894e-05, 1.0171514451757102e-21, 2.4791791730649493e-08, 4.764723144035088e-06, 0.0011091399937868118, 8.503096095074589e-14, 0.007965074852108955, 2.867164994313498e-08, 8.559239358874038e-05, 5.805446591121075e-14, 0.000249513570452109, 1.8039070710074157e-05, 1.2116474490397877e-09, 4.764858507755321e-10, 0.19458331167697906, 0.03818589821457863, 0.17832297086715698, 2.3084856248800634e-10, 3.0443703114002574e-11, 1.6708211203475543e-10, 0.06018122285604477, 0.00239556422457098

...

Training cluster_1: 100%|██████████| 40/40 [00:20<00:00,  1.97it/s]


SYNC Weights:[3.6156768696713206e-09, 0.15157355218828075, 2.407400156934541e-06, 0.00437756581082615, 0.023812564724356215, 2.463281910993613e-07, 4.567548762331003e-09, 0.019951832509857905, 0.15815440125455343, 0.054392023394552985, 0.0032480694214531993, 0.03450451180019956, 2.45693279739197e-17, 0.021016370835000887, 0.01583427820757094, 0.0036921597506434133, 1.1776918710699487e-13, 0.00023196335714871616, 0.17171125691795255, 0.0004079039073783329, 0.08022388266153231, 4.2407640316849826e-13, 7.41039252300914e-08, 0.00022441504453499556, 0.02136536513652872, 0.000195945578872394, 2.159392117966059e-09, 0.00218578476189039, 0.005772006408399909, 0.029813135661771125, 0.0034615809413151293, 0.00021660344238541888, 9.110362367628102e-08, 0.15846405216437617, 0.010070545071724949, 8.981935568881614e-08, 4.590412308279879e-18, 0.005362233804525241, 0.019287721844079047, 0.00044535258276436393] 
Saved model: results/lstm/cluster_1/lstm_cluster_1_round_2_kr_softmax.pt
 Processing clust

Training cluster_2: 100%|██████████| 26/26 [00:13<00:00,  1.96it/s]


SYNC Weights:[0.698261345828447, 0.04900568871561125, 1.6756162485705287e-10, 7.963161391747661e-07, 3.906903720138182e-06, 8.545806816431385e-19, 4.499495121253197e-08, 0.037946046213360604, 6.865081759945577e-17, 5.9456088215139635e-06, 1.2072079688416604e-06, 0.19629194972098077, 0.0011923980374804112, 3.374326089151393e-09, 4.764545305029606e-05, 4.284316816328194e-06, 0.002093788033904626, 1.489574633047522e-06, 8.68600797874767e-09, 4.414492152154982e-08, 9.458805276386944e-06, 0.01507950826014939, 1.606627693386796e-20, 1.1484824625044246e-05, 3.9695404388524184e-05, 3.2524242448407194e-06] 
Saved model: results/lstm/cluster_2/lstm_cluster_2_round_2_kr_softmax.pt
\Round 3/10
 Processing cluster_0 with 537 clients
Sampling 80 Clients


Training cluster_0:  48%|████▊     | 38/80 [00:19<00:20,  2.04it/s]

### Kuramoto Norm FedAvg

In [ ]:
# DEVICE
# Main experiment loop
for model_name in MODEL_NAMES:
    print(f"Starting experiments for model: {model_name}")

    # Initialize per-cluster model weights
    cluster_models = {}
    cluster_weights = {}

    for cluster_name in CLUSTERS:
        model = model_fn(model_name).to(DEVICE)
        cluster_models[cluster_name] = model
        cluster_weights[cluster_name] = get_weights(model)

    for rnd in range(NUM_ROUNDS):
        print(f"\Round {rnd+1}/{NUM_ROUNDS}")

        for cluster_name, client_ids in CLUSTERS.items():
            print(f" Processing {cluster_name} with {len(client_ids)} clients")

            # Sample a fraction of clients from the cluster
            sampled_clients = random.sample(client_ids, k=int(CLIENT_FRAC * len(client_ids)))
            print(f"Sampling {len(sampled_clients)} Clients")
            local_weights = []

            for cid in tqdm(sampled_clients, desc=f"Training {cluster_name}"):
                local_model = model_fn(model_name).to(DEVICE)
                set_weights(local_model, cluster_weights[cluster_name])

                train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)
                updated_weights, fin_loss = train_model(
                    local_model, train_loader,
                    device=DEVICE,
                    learning_rate=LR,
                    loss_fn=None,
                    optimizer_class=optim.Adam,
                    epochs=LOCAL_EPOCHS
                )
                local_weights.append(updated_weights)

            # Aggregate and update cluster model
            updated_cluster_weights = sync_aggregate_softmax(cluster_weights[cluster_name],local_weights)
            set_weights(cluster_models[cluster_name], updated_cluster_weights)
            cluster_weights[cluster_name] = updated_cluster_weights

            # Save checkpoint
            ckpt_path = os.path.join("results", model_name, cluster_name, f"{model_name}_{cluster_name}_round_{rnd+1}_kr_norm.pt")
            torch.save(cluster_models[cluster_name].state_dict(), ckpt_path)
            print(f"Saved model: {ckpt_path}")


Starting experiments for model: lstm
\Round 1/10
 Processing cluster_0 with 537 clients
Sampling 80 Clients


Training cluster_0: 100%|██████████| 80/80 [00:45<00:00,  1.77it/s]


SYNC Weights:[0.011088981293141842, 0.0005276454612612724, 0.009006999433040619, 0.0010065635433420539, 0.02894829772412777, 0.0005409520817920566, 0.002419608412310481, 0.03965654596686363, 0.01275147870182991, 0.0060289776884019375, 0.0029398140031844378, 0.026724137365818024, 0.0005258911987766623, 0.001337263616733253, 0.005358441732823849, 0.007517630700021982, 0.031638406217098236, 0.023924464359879494, 0.009779023006558418, 0.012516401708126068, 0.010698819532990456, 0.008907420560717583, 0.02152569591999054, 0.018377719447016716, 0.000958866614382714, 0.015023786574602127, 0.00016531653818674386, 0.01078857947140932, 0.00011684942728606984, 0.014073467813432217, 0.017146386206150055, 0.010822692885994911, 0.026125604286789894, 0.0010200496762990952, 0.020381156355142593, 0.0013889539986848831, 0.015629161149263382, 0.013110995292663574, 0.0012778245145455003, 0.035792287439107895, 4.69821497972589e-05, 0.00017756293527781963, 0.01291549764573574, 0.0010281517170369625, 0.006933

Training other: 100%|██████████| 64/64 [00:35<00:00,  1.78it/s]


SYNC Weights:[0.07462282478809357, 0.00028802879387512803, 0.00030799463274888694, 0.0001894784509204328, 0.1856829822063446, 0.0013856367440894246, 0.006020246539264917, 0.00015760045789647847, 0.0006232321029528975, 0.034824274480342865, 1.9031719489248644e-07, 5.8000339777208865e-05, 0.001954455394297838, 0.0020411398727446795, 0.0001238178665516898, 3.321071562822908e-05, 0.0008578212582506239, 0.3715418577194214, 0.00020522030536085367, 0.00010857664165087044, 3.190814459230751e-05, 3.9604579797014594e-05, 1.5161933788476745e-06, 0.0057764663361012936, 0.07607322186231613, 2.0615650300152139e-13, 5.689662430086173e-07, 0.0014188580680638552, 0.00011375695612514392, 1.9329167599835273e-08, 1.045159070400814e-09, 1.5019485317679937e-06, 0.0910891517996788, 1.0546632438490633e-05, 0.0007252946961671114, 4.5648281229659915e-05, 3.2138357865816106e-14, 0.0001444595109205693, 0.004598150495439768, 1.1947526301625544e-09, 1.9924131393502853e-21, 0.0003511750837787986, 0.00096852390561252

...

Training cluster_1: 100%|██████████| 40/40 [00:22<00:00,  1.75it/s]


SYNC Weights:[0.014699674648086965, 1.2582828521198813e-05, 0.005710630953538031, 0.0033764451547953073, 0.017942139873202076, 0.02504463671239415, 1.686642816380579e-06, 0.00022320883287946835, 0.004667675165671961, 0.09030122976228626, 0.035755672242407624, 3.565133092615024e-05, 0.029339260513795238, 0.023507350320063063, 0.033774325282017396, 0.008941449912009156, 0.019018252616502175, 0.020708851440195984, 0.06288285927977907, 0.0330256393156277, 0.05824745733733166, 0.04660567300817488, 0.019136486750623755, 9.209454891748386e-05, 0.029975622930677744, 0.011862491619755135, 0.008600741231225307, 0.017212682929970396, 0.12140029734910307, 0.004977366248882371, 2.1251402384250566e-06, 0.021833262255366373, 0.024432688008199088, 0.02474519205273004, 0.024642845037726402, 0.057534864629546725, 0.03536397393604041, 5.6922613866958825e-06, 0.0032554282490209457, 0.061103790433564766] 
Saved model: results/gru/cluster_1/gru_cluster_1_round_4_kr_norm.pt
 Processing cluster_2 with 179 cli

Training cluster_2: 100%|██████████| 26/26 [00:14<00:00,  1.75it/s]


SYNC Weights:[8.175430601066606e-105, 2.908272360250939e-163, 6.369173081823506e-151, 3.4030753823157673e-74, 2.4022690517037835e-55, 3.1303208597061415e-40, 5.595839008732146e-134, 2.9262235613824243e-134, 1.0279549655462605e-129, 2.415450556482585e-129, 2.204440335747829e-75, 1.361276129976628e-134, 1.4743179206554549e-174, 7.143436488861361e-142, 1.127590788455467e-75, 2.547726016629489e-133, 4.217097925158844e-146, 3.804723652910579e-158, 0.9999999900000002, 1.2071680837615078e-56, 3.409698392356127e-96, 5.321674145677865e-103, 5.067189627587751e-209, 1.3015043786000759e-101, 7.876063959569107e-267, 4.243257671402249e-201] 
Saved model: results/gru/cluster_2/gru_cluster_2_round_4_kr_norm.pt
\Round 5/10
 Processing cluster_0 with 537 clients
Sampling 80 Clients


Training cluster_0:  76%|███████▋  | 61/80 [00:35<00:10,  1.78it/s]

### Clustered

### Kuramoto FedAvg

In [23]:


# Config
# List of models to experiment with
MODEL_NAMES = ["lstm", "gru", "moe_lstm", "moe_gru"]

# Config
NUM_CLIENTS = 1500
CLIENT_FRAC = 0.15
NUM_ROUNDS = 10
LOCAL_EPOCHS = 10
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_FILE ="train_cleaned_reindex.feather" # "meter_0_data_cleaned.feather"


### Diff-Aware Fed Avg

In [12]:
class TimeSeriesDifficultyWeight:
    def __init__(self, num_clients, accumulate_iters=20):
        self.num_clients = num_clients
        self.last_loss = torch.ones(num_clients).float().to(DEVICE)
        self.learn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.unlearn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.ema_difficulty = torch.ones(num_clients).float().to(DEVICE)
        self.accumulate_iters = accumulate_iters

    def update(self, cid: int, loss_history: List[float]) -> float:
        """
        Update difficulty based on loss trend for a client.
        Expects a list of per-epoch losses.
        """
        current_loss = torch.tensor(loss_history[-1], dtype=torch.float32).to(DEVICE)
        previous_loss = self.last_loss[cid]
        delta = current_loss - previous_loss
        ratio = torch.log((current_loss + 1e-8) / (previous_loss + 1e-8))

        learn = torch.where(delta < 0, -delta * ratio, torch.tensor(0.0, device=current_loss.device))
        unlearn = torch.where(delta >= 0, delta * ratio, torch.tensor(0.0, device=current_loss.device))

        # EMA update
        momentum = (self.accumulate_iters - 1) / self.accumulate_iters
        self.learn_score[cid] = momentum * self.learn_score[cid] + (1 - momentum) * learn
        self.unlearn_score[cid] = momentum * self.unlearn_score[cid] + (1 - momentum) * unlearn

        # Difficulty score
        diff_ratio = (self.unlearn_score[cid] + 1e-8) / (self.learn_score[cid] + 1e-8)
        difficulty = diff_ratio #torch.pow(diff_ratio, 1 / 5)

        # Smooth difficulty over rounds
        self.ema_difficulty[cid] = momentum * self.ema_difficulty[cid] + (1 - momentum) * difficulty

        self.last_loss[cid] = current_loss
        return self.ema_difficulty[cid].item()

    def get_normalized_weights(self, client_ids: List[int]) -> List[float]:
        weights = [self.ema_difficulty[cid].item() for cid in client_ids]
        total = sum(weights)
        if total == 0:
            return [1.0 / len(client_ids)] * len(client_ids)
        return [w / total for w in weights]


In [15]:
1/np.array([0.1,0.2,2])

array([10. ,  5. ,  0.5])

In [18]:
MODEL_NAMES = ['gru','lstm']

In [19]:
for model_name in MODEL_NAMES:
    difficulty_tracker = TimeSeriesDifficultyWeight(num_clients=NUM_CLIENTS,accumulate_iters=10)
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        sampled_clients = random.sample(range(NUM_CLIENTS), int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []
        difficulty_scores = []

        for cid in tqdm(sampled_clients):
            model = model_fn(model_name).to(DEVICE)
            set_weights(model, global_weights)
            train_loader, _ = load_energy_data_feather(cid, filepath=DATA_FILE)
            updated_weights, loss_history = train_model(
                model, train_loader,
                device=DEVICE, learning_rate=LR,
                loss_fn=None, optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

            # Update difficulty
            difficulty = difficulty_tracker.update(cid, loss_history)
            difficulty_scores.append(difficulty)

        # Normalize difficulty scores
        normalized_weights = difficulty_tracker.get_normalized_weights(sampled_clients)

        # Difficulty-aware weighted aggregation
        global_weights = average_weights(local_weights, client_weights=normalized_weights)
        set_weights(global_model, global_weights)

        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_diff_lr2.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: gru


100%|██████████| 211/211 [01:35<00:00,  2.21it/s]


Saved global model to results/gru/gru_round_1_diff_lr2.pt


100%|██████████| 211/211 [01:33<00:00,  2.25it/s]


Saved global model to results/gru/gru_round_2_diff_lr2.pt


...

Saved global model to results/lstm/lstm_round_48_diff_lr2.pt


100%|██████████| 211/211 [01:34<00:00,  2.24it/s]


Saved global model to results/lstm/lstm_round_49_diff_lr2.pt


100%|██████████| 211/211 [01:33<00:00,  2.26it/s]

Saved global model to results/lstm/lstm_round_50_diff_lr2.pt


In [ ]:
class TimeSeriesDifficultyWeightRev:
    def __init__(self, num_clients, accumulate_iters=20):
        self.num_clients = num_clients
        self.last_loss = torch.ones(num_clients).float().to(DEVICE)
        self.learn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.unlearn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.ema_difficulty = torch.ones(num_clients).float().to(DEVICE)
        self.accumulate_iters = accumulate_iters

    def update(self, cid: int, loss_history: List[float]) -> float:
        """
        Update difficulty based on loss trend for a client.
        Expects a list of per-epoch losses.
        """
        current_loss = torch.tensor(loss_history[-1], dtype=torch.float32).to(DEVICE)
        previous_loss = self.last_loss[cid]
        delta = current_loss - previous_loss
        ratio = torch.log((current_loss + 1e-8) / (previous_loss + 1e-8))

        learn = torch.where(delta < 0, -delta * ratio, torch.tensor(0.0, device=current_loss.device))
        unlearn = torch.where(delta >= 0, delta * ratio, torch.tensor(0.0, device=current_loss.device))

        # EMA update
        momentum = (self.accumulate_iters - 1) / self.accumulate_iters
        self.learn_score[cid] = momentum * self.learn_score[cid] + (1 - momentum) * learn
        self.unlearn_score[cid] = momentum * self.unlearn_score[cid] + (1 - momentum) * unlearn

        # Difficulty score
        diff_ratio = (self.unlearn_score[cid] + 1e-8) / (self.learn_score[cid] + 1e-8)
        difficulty = diff_ratio #torch.pow(diff_ratio, 1 / 5)

        # Smooth difficulty over rounds
        self.ema_difficulty[cid] = momentum * self.ema_difficulty[cid] + (1 - momentum) * difficulty

        self.last_loss[cid] = current_loss
        return self.ema_difficulty[cid].item()

    def get_normalized_weights(self, client_ids: List[int]) -> List[float]:
        weights = [1/self.ema_difficulty[cid].item() for cid in client_ids]
        total = sum(weights)
        if total == 0:
            return [1.0 / len(client_ids)] * len(client_ids)
        return [w / total for w in weights]


In [20]:
for model_name in MODEL_NAMES:
    difficulty_tracker = TimeSeriesDifficultyWeightRev(num_clients=NUM_CLIENTS,accumulate_iters=20)
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        sampled_clients = random.sample(range(NUM_CLIENTS), int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []
        difficulty_scores = []

        for cid in tqdm(sampled_clients):
            model = model_fn(model_name).to(DEVICE)
            set_weights(model, global_weights)
            train_loader, _ = load_energy_data_feather(cid, filepath=DATA_FILE)
            updated_weights, loss_history = train_model(
                model, train_loader,
                device=DEVICE, learning_rate=LR,
                loss_fn=None, optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS
            )
            local_weights.append(updated_weights)

            # Update difficulty
            difficulty = difficulty_tracker.update(cid, loss_history)
            difficulty_scores.append(difficulty)

        # Normalize difficulty scores
        normalized_weights = difficulty_tracker.get_normalized_weights(sampled_clients)

        # Difficulty-aware weighted aggregation
        global_weights = average_weights(local_weights, client_weights=normalized_weights)
        set_weights(global_model, global_weights)

        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_diff_rev_lr2.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: gru


100%|██████████| 211/211 [01:34<00:00,  2.24it/s]


Saved global model to results/gru/gru_round_1_diff_rev_lr2.pt


100%|██████████| 211/211 [01:35<00:00,  2.21it/s]


Saved global model to results/gru/gru_round_2_diff_rev_lr2.pt


...

Saved global model to results/lstm/lstm_round_48_diff_rev_lr2.pt


100%|██████████| 211/211 [01:43<00:00,  2.04it/s]


Saved global model to results/lstm/lstm_round_49_diff_rev_lr2.pt


100%|██████████| 211/211 [01:42<00:00,  2.06it/s]

Saved global model to results/lstm/lstm_round_50_diff_rev_lr2.pt


In [26]:
for model_name in MODEL_NAMES:
    difficulty_tracker = TimeSeriesDifficultyWeight(num_clients=NUM_CLIENTS)
    print(f"Starting experiment with model: {model_name}")
    # Initialize cluster-specific models and difficulty trackers
    cluster_models = {}
    cluster_weights = {}
    cluster_difficulty = {}

    for cluster_name in CLUSTERS:
        model = model_fn(model_name).to(DEVICE)
        weights = get_weights(model)
        difficulty_tracker = TimeSeriesDifficultyWeight(num_clients=NUM_CLIENTS)

        cluster_models[cluster_name] = model
        cluster_weights[cluster_name] = weights
        cluster_difficulty[cluster_name] = difficulty_tracker

    # Training loop
    for rnd in range(NUM_ROUNDS):
        print(f"\nRound {rnd + 1}/{NUM_ROUNDS}")

        for cluster_name, client_ids in CLUSTERS.items():
            print(f"Training {cluster_name} with {len(client_ids)} clients")

            # Sample a subset of clients in this cluster
            sampled_clients = random.sample(client_ids, k=int(CLIENT_FRAC * len(client_ids)))
            local_weights = []

            difficulty_tracker = cluster_difficulty[cluster_name]
            weights_before = cluster_weights[cluster_name]
            model_template = cluster_models[cluster_name]

            for cid in tqdm(sampled_clients, desc=f"Cluster {cluster_name}"):
                local_model = model_fn(model_name).to(DEVICE)
                set_weights(local_model, weights_before)

                train_loader, _ = load_energy_data_feather(cid, filepath=DATA_FILE)
                updated_weights, loss_history = train_model(
                    local_model, train_loader,
                    device=DEVICE, learning_rate=LR,
                    loss_fn=None, optimizer_class=optim.Adam,
                    epochs=LOCAL_EPOCHS
                )
                local_weights.append(updated_weights)

                # Update difficulty tracking
                difficulty_tracker.update(cid, loss_history)

            # Get difficulty-based normalized weights
            normalized_weights = difficulty_tracker.get_normalized_weights(sampled_clients)

            # Weighted aggregation
            new_weights = average_weights(local_weights, client_weights=normalized_weights)

            # Update cluster model
            set_weights(model_template, new_weights)
            cluster_weights[cluster_name] = new_weights

            # Save checkpoint
            ckpt_dir = os.path.join("results", model_name, cluster_name)
            os.makedirs(ckpt_dir, exist_ok=True)
            ckpt_path = os.path.join(ckpt_dir, f"{model_name}_{cluster_name}_round_{rnd + 1}_diff.pt")
            torch.save(model_template.state_dict(), ckpt_path)
            print(f"Saved model to {ckpt_path}")


Starting experiment with model: lstm

Round 1/10
Training cluster_0 with 537 clients


Cluster cluster_0:   0%|          | 0/80 [00:00<?, ?it/s]

Cluster cluster_0: 100%|██████████| 80/80 [00:25<00:00,  3.14it/s]


Saved model to results/lstm/cluster_0/lstm_cluster_0_round_1_diff.pt
Training other with 428 clients


Cluster other: 100%|██████████| 64/64 [00:20<00:00,  3.12it/s]


...

Saved model to results/moe_gru/other/moe_gru_other_round_10_diff.pt
Training cluster_1 with 269 clients


Cluster cluster_1: 100%|██████████| 40/40 [00:16<00:00,  2.49it/s]


Saved model to results/moe_gru/cluster_1/moe_gru_cluster_1_round_10_diff.pt
Training cluster_2 with 179 clients


Cluster cluster_2: 100%|██████████| 26/26 [00:10<00:00,  2.48it/s]

Saved model to results/moe_gru/cluster_2/moe_gru_cluster_2_round_10_diff.pt


### Diff-Sync FedAvg